# general notes

note that the points/centres/target are stored in nx3 numpy arrays where the first colum is the number, the second and third the coordinates. This should be maintained, as this is compatible with Yuki's transformation code. 

you need to add the target information as well

the end result is a list of lists, potPointMatch, which is the same length as goodIdx. Use goodIdx to transform to the cobra number. The contents of the list is a single element list which contains the index of the point that matches the cobra (in the array points), or an empty list if there is no match. 

The code cannot assign the same point to more than one cobra


# imports

Note that you need to fix the paths for your system

In [244]:
%matplotlib notebook

#python package imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from astropy.io import fits
import sys

#pfi system imports
prefix="/Users/karr/Science/PFS/cobraData/Converge21/"
#sys.path.append('/Users/chyan/Documents/workspace/ics_cobraCharmer/python/ics/cobraCharmer')
sys.path.insert(1, "/Users/karr/Science/PFS/pfsPackages/ics_cobraCharmer/python/procedures/moduleTest/")
sys.path.insert(1, "/Users/karr/Science/PFS/pfsPackages/ics_cobraCharmer/python/ics/cobraCharmer/")
sys.path.insert(1, "/Users/karr/Science/PFS/cobraData/")
import pfi as pfi 

#my imports
import fibreID as fi
import centroid


# set variables

change broken and xmlFile to current values

In [245]:



prefix="/Users/karr/Science/PFS/cobraData/Full2D/"
xmlFile=prefix+'20210219_002/output/ALL_new.xml'
broken=[47,193,347,361,754,799,821,853,949,1150,1208,1210,1520,1580,1637,1653,1724,1825,1836,1882,1903,2053,2352,2380]


# call routines for variables

getCalibBench gets the centers, arm lengths, etc, and makes the goodIdx variable

standardParameters loads the centroiding parameters

These two only need to be done once per configuration. 


In [246]:
reload(fi)
#temporary routine to import information from xml file
pfic,centers,centersAll,goodIdx,arms,armsAll,nCobras,nCobrasAll=fi.getCalibBench(xmlFile,broken)

#temporary routine to get the centroiding variables
sigmaFind,sigmaCent,sigmaThresh,boxFind,boxCent,nMin,nMax,maxIt,fwhmX,fwhmY=fi.standardParameters()



/Users/karr/Science/PFS/pfsPackages/ics_cobraCharmer/python/ics/cobraCharmer/pfiDesign.py:271: RuntimeWarning: divide by zero encountered in true_divide
  self.S1Nm[i] = angularStep / np.array(list(map(float, slowJoint1Rev)))
2021-03-31T14:03:46.915 20 pfi        pfi.py:102 load cobra model from [PosixPath('/Users/karr/Science/PFS/cobraData/Full2D/20210219_002/output/ALL_new.xml')]


# calculate threshold for centroiding

only need to do once per configuration, until the lighting conditions change

In [247]:
#load iamge
image=fits.getdata("testdata/PFSC00009169.fits")

threshFind,threshCent=fi.getThresh(image,centers,sigmaFind,sigmaCent,sigmaThresh)
print(threshFind,threshCent)


183.1120411058803 56.95786471991921


# some pre-calculations for fibre identification

makeAdjacentList only needs to be re-done when the centre positions change

makeSpirals only needs to be done once

In [47]:
#list of adjacent cobras, mostly important with spots
adjacentCobras=fi.makeAdjacentList(centers,arms)
#spiral pattern for searching around spots
spiralX,spiralY=fi.makeSpirals(6,15)


# centroid

The code below here is done once for each movement/image

In [249]:

image=fits.getdata("20210219_009/data/PFSC00009169.fits")

#centroid
points,nPoints=fi.getCentroids(image,threshFind,threshCent,boxFind,boxCent,nMin,nMax,maxIt)


(2389, 2370) (2370,)


# identify

In [40]:
# set up the bookkeeping variables
aCobras,unaCobras,aPoints,unaPoints,potCobraMatch,potPointMatch=fi.prepWork(points,nPoints,nCobras,centers,arms,goodIdx)
# first pass through identifications
aCobras,unaCobras,aPoints,unaPoints,potCobraMatch,potPointMatch,anyChange=fi.firstPass(aCobras,unaCobras,aPoints,unaPoints,potCobraMatch,potPointMatch)
#second pass; ambigious cobras ar assigned to the nearest point to the target 
aCobras,unaCobras,aPoints,unaPoints,potCobraMatch,potPointMatch=fi.secondPassBench(aCobras,unaCobras,aPoints,unaPoints,potCobraMatch,potPointMatch,points,targets)



(2389, 2370) (2370,)
